
# 📊 Sales Funnel Metrics for GTM & Pricing Analytics

This section demonstrates my ability to calculate key sales funnel and go-to-market (GTM) efficiency metrics using SQL. These metrics are fundamental for forecasting, evaluating sales performance, and informing pricing strategy.

The calculation for these metrics typically involves the concept of **conversion rate**, which is generally:
$$\text{Conversion Rate} = \frac{\text{Number of entities moving to the next stage}}{\text{Number of entities in the current stage}}$$

### Table of Contents:
1. Lead-to-Opportunity Conversion Rate (L2O) 
2. Opportunity-to-Win Conversion Rate (O2W) / Win Rate 
3. Sales Cycle Length (Time to Close) 
4. Average Deal Size (or Average Annual Contract Value - ACV) 
5. Pipeline Coverage Ratio


***

## 1. Lead-to-Opportunity Conversion Rate (L2O)

Measures the efficiency of converting raw leads into qualified opportunities.

| Detail | Description |
| :--- | :--- |
| **Why it Matters** | Gauges the effectiveness of **marketing efforts** and initial lead qualification (SDR/BDR teams). |
| **Formula** | $$\frac{\text{COUNT(Opportunities Created)}}{\text{COUNT(Total Leads Created)}}$$ |
| **Required Data Fields** | `lead_id`, `is_opportunity_created` (Boolean/Flag), `lead_creation_date` |


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW leads 
USING CSV
OPTIONS (
  path "file:/Workspace/Users/olesia.tankersley@thermofisher.com/Go-To-Market-Analytics/lead_funnel_data.csv",
  header "true",
  inferSchema "true" );

In [0]:
 %sql
select * from leads limit 5

lead_id,lead_creation_date,is_opportunity_created,opportunity_creation_date
L-1,2025-01-01,1,2025-01-02
L-10,2025-01-19,1,2025-01-24
L-100,2025-06-18,0,null
L-101,2025-06-19,0,null
L-102,2025-06-20,1,2025-06-25


In [0]:
#alternative way to inspect data
query = "SELECT * FROM leads LIMIT 5"
df = spark.sql(query)
display(df)

lead_id,lead_creation_date,is_opportunity_created,opportunity_creation_date
L-1,2025-01-01,1,2025-01-02
L-10,2025-01-19,1,2025-01-24
L-100,2025-06-18,0,null
L-101,2025-06-19,0,null
L-102,2025-06-20,1,2025-06-25


In [0]:
%sql 
select date_part('month', lead_creation_date) as month, count(distinct lead_id) as num_leads
      ,count(distinct case when is_opportunity_created = 1 then lead_id else null end) as num_opportunities from leads
group by date_part('month', lead_creation_date)
order by date_part('month', lead_creation_date)

month,num_leads,num_opportunities
1,15,6
2,15,7
3,20,8
4,20,6
5,20,10
6,20,9
7,18,8
8,12,3
9,15,6
10,20,8


In [0]:
%sql 
with CTE as (

select date_part('month', lead_creation_date) as month, count(distinct lead_id) as num_leads
      ,count(distinct case when is_opportunity_created = 1 then lead_id else null end) as num_opportunities from leads
group by date_part('month', lead_creation_date)
order by date_part('month', lead_creation_date)

)
select month, round(num_opportunities/num_leads,2) as L2O 
from CTE

month,L2O
1,0.4
2,0.47
3,0.4
4,0.3
5,0.5
6,0.45
7,0.44
8,0.25
9,0.4
10,0.4


***
## 2. Opportunity-to-Win Conversion Rate (Win Rate)

Measures the percentage of closed opportunities that result in a new customer (a "win").

| Detail | Description |
| :--- | :--- |
| **Why it Matters** | Core indicator of **sales team effectiveness** and market competitiveness (product/pricing). |
| **Formula** | $$\frac{\text{COUNT(Won Deals)}}{\text{COUNT(Total Closed Opportunities)}}$$ |
| **Required Data Fields** | `opportunity_id`, `deal_status` (must include 'Won' and 'Lost'), `closed_date` |



In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW opportunity 
USING CSV
OPTIONS (
  path "file:/Workspace/Users/olesia.tankersley@thermofisher.com/Go-To-Market-Analytics/opportunity_funnel_data.csv",
  header "true",
  inferSchema "true" );

In [0]:
%sql
SELECT * FROM opportunity LIMIT 5

opportunity_id,opportunity_creation_date,deal_value,closed_date,deal_status
OPP-202501-01,2025-01-01,15000,2025-01-15,Won
OPP-202501-02,2025-01-02,25000,2025-01-20,Lost
OPP-202501-03,2025-01-03,30000,2025-01-25,Won
OPP-202501-04,2025-01-05,12000,2025-01-18,Lost
OPP-202501-05,2025-01-07,40000,2025-02-01,Won


In [0]:
%sql
select date_part('month', opportunity_creation_date) as month
,count(distinct opportunity_id)                      as num_opportunities
,count(distinct case when deal_status = 'Won' then opportunity_id else null end) as num_wins 
from opportunity
group by date_part('month', opportunity_creation_date)
order by date_part('month', opportunity_creation_date)

month,num_opportunities,num_wins
1,20,8
2,20,6
3,18,7
4,21,6
5,21,8
6,15,5
7,19,6
8,26,9
9,17,7
10,23,6


In [0]:
%sql
with CTE as (
  
  select date_part('month', opportunity_creation_date) as month
  ,count(distinct opportunity_id)                      as num_opportunities
  ,count(distinct case when deal_status = 'Won' then opportunity_id else null end) as num_wins 
  from opportunity
  group by date_part('month', opportunity_creation_date)
  order by date_part('month', opportunity_creation_date)

)

select month, round(num_wins/num_opportunities, 2) as O2W
from CTE

month,O2W
1,0.4
2,0.3
3,0.39
4,0.29
5,0.38
6,0.33
7,0.32
8,0.35
9,0.41
10,0.26


***
## 3. Sales Cycle Length (Time to Close)

Measures the average duration (in days) from the opportunity creation to its closure.

| Detail | Description |
| :--- | :--- |
| **Why it Matters** | Shorter cycles improve forecasting accuracy and **cash flow**. Informs sales process optimization. |
| **Formula** | $$\text{AVG}(\text{Closed Date} - \text{Opportunity Creation Date})$$ |
| **Required Data Fields** | `opportunity_id`, `opportunity_creation_date`, `closed_date`, `deal_status` (Filter for closed deals only) |


In [0]:
%sql
SELECT * FROM opportunity LIMIT 5

opportunity_id,opportunity_creation_date,deal_value,closed_date,deal_status
OPP-202501-01,2025-01-01,15000,2025-01-15,Won
OPP-202501-02,2025-01-02,25000,2025-01-20,Lost
OPP-202501-03,2025-01-03,30000,2025-01-25,Won
OPP-202501-04,2025-01-05,12000,2025-01-18,Lost
OPP-202501-05,2025-01-07,40000,2025-02-01,Won


In [0]:
%sql
select date_part('month', opportunity_creation_date) as month 
,round(avg(date_diff(closed_date,opportunity_creation_date)),0) as num_days
from opportunity
group by date_part('month', opportunity_creation_date)
order by date_part('month', opportunity_creation_date)

month,num_days
1,26.0
2,30.0
3,30.0
4,31.0
5,38.0
6,29.0
7,31.0
8,48.0
9,33.0
10,41.0


In [0]:
%sql
select date_part('month', opportunity_creation_date) as month 
,round(avg(date_diff(closed_date,opportunity_creation_date)),0) as num_days
from opportunity
where deal_status = 'Won'
group by date_part('month', opportunity_creation_date)
order by date_part('month', opportunity_creation_date)

month,num_days
1,26.0
2,30.0
3,31.0
4,33.0
5,35.0
6,30.0
7,34.0
8,42.0
9,30.0
10,39.0


***
## 4. Average Deal Size (Average ACV)

The average Annual Contract Value (ACV) of all closed-won deals within a period.

| Detail | Description |
| :--- | :--- |
| **Why it Matters** | A primary **pricing analytic**. Tracks shifts in customer segmentation and success of pricing tiers/upsells. |
| **Formula** | $$\frac{\text{SUM(Deal Value of Won Deals)}}{\text{COUNT(Won Deals)}}$$ |
| **Required Data Fields** | `opportunity_id`, `deal_value` (or `acv`), `deal_status` (Must be 'Won'), `closed_date` |

In [0]:
%sql
SELECT * FROM opportunity LIMIT 5

opportunity_id,opportunity_creation_date,deal_value,closed_date,deal_status
OPP-202501-01,2025-01-01,15000,2025-01-15,Won
OPP-202501-02,2025-01-02,25000,2025-01-20,Lost
OPP-202501-03,2025-01-03,30000,2025-01-25,Won
OPP-202501-04,2025-01-05,12000,2025-01-18,Lost
OPP-202501-05,2025-01-07,40000,2025-02-01,Won


In [0]:
%sql
select date_part('month', opportunity_creation_date) as month
,round(avg(deal_value),0) as avg_won_deal_value
from opportunity
where deal_status = 'Won'
group by date_part('month', opportunity_creation_date)
order by date_part('month', opportunity_creation_date)

month,avg_won_deal_value
1,26750.0
2,24333.0
3,25429.0
4,25333.0
5,26625.0
6,27200.0
7,19833.0
8,26444.0
9,27000.0
10,25333.0


In [0]:
%sql
--alternative way to filter out the data
select date_part('month', opportunity_creation_date) as month
,cast(avg(deal_value) filter (where deal_status = 'Won') as decimal(22,0)) as avg_won_deal_value
from opportunity
group by date_part('month', opportunity_creation_date)
order by date_part('month', opportunity_creation_date)

month,avg_won_deal_value
1,26750
2,24333
3,25429
4,25333
5,26625
6,27200
7,19833
8,26444
9,27000
10,25333


***
## 5. Pipeline Coverage Ratio

Compares the value of the active sales pipeline against the revenue target for a period.

| Detail | Description |
| :--- | :--- |
| **Why it Matters** | A **GTM health metric** that informs resource allocation and risk. A common target is 3x to 4x coverage. |
| **Formula** | $$\frac{\text{SUM(Value of Active Opportunities)}}{\text{Sales Revenue Target}}$$ |
| **Required Data Fields** | `opportunity_id`, `deal_value` (Forecasted), `stage` (Must exclude 'Closed' stages), **External Data: Revenue Target** |
| **SQL Concept** | `SUM(deal_value) FILTER (WHERE stage NOT IN ('Closed Won', 'Closed Lost')) / [Target Variable]` |

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW pipeline 
USING CSV
OPTIONS (
  path "file:/Workspace/Users/olesia.tankersley@thermofisher.com/Go-To-Market-Analytics/active_pipeline_data.csv",
  header "true",
  inferSchema "true" );

In [0]:
%sql
SELECT * FROM pipeline LIMIT 5

opportunity_id,current_stage,forecasted_deal_value,estimated_close_date
PIPE-2025-01,Negotiation,55000,2025-10-15
PIPE-2025-02,Proposal,28000,2025-10-25
PIPE-2025-03,Discovery,17000,2025-11-01
PIPE-2025-04,Negotiation,48000,2025-11-10
PIPE-2025-05,Discovery,22000,2025-11-15


In [0]:
%sql
select date_part('year', estimated_close_date) as year, date_part('month', estimated_close_date) as month,
round(sum(forecasted_deal_value),0) as pipeline_value
from pipeline
group by date_part('year', estimated_close_date), date_part('month', estimated_close_date)
order by date_part('year', estimated_close_date), date_part('month', estimated_close_date)

year,month,pipeline_value
2025,10,83000
2025,11,252000
2025,12,424000
2026,1,556000
2026,2,545000
2026,3,756000
2026,4,464000
2026,5,798000
2026,6,632000
2026,7,430000


In [0]:
%sql
with CTE as (SELECT 400000 as target)
select date_part('year', estimated_close_date) as year, date_part('month', estimated_close_date) as month,
round(sum(forecasted_deal_value)/avg(target),2) as coverage_ratio
from pipeline
cross join CTE
where pipeline.current_stage not like 'Closed'
group by date_part('year', estimated_close_date), date_part('month', estimated_close_date)
order by date_part('year', estimated_close_date), date_part('month', estimated_close_date)

year,month,coverage_ratio
2025,10,0.21
2025,11,0.63
2025,12,1.06
2026,1,1.39
2026,2,1.36
2026,3,1.89
2026,4,1.16
2026,5,2.0
2026,6,1.58
2026,7,1.08
